In [ ]:
# Dependencies and Setup
import openweathermapy as ow
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time

# Import API key
from config import api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

#Outline stylesheet to use for plots
plt.style.use('ggplot')

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).

In [ ]:
# Create a settings dictionary with your API key and preferred units
settings = {"units": "imperial", "appid": api_key}

In [ ]:
# Store base url
url = "http://api.openweathermap.org/data/2.5/weather?"

# Set up lists to hold reponse info
clouds = []
country = []
humidity = []
lat = []
lon = []
max_temp = []
wind_speed = []

# Loop through the list of cities and perform a request for data on each
for city in cities:
    print(f"Processing request for {city}")
    response = requests.get(url + "appid=" + api_key + "&q=" + city +"&units=imperial").json()
    try:
        clouds.append(response["clouds"]["all"])
    except KeyError:
        clouds.append("NaN")
    try:
        country.append(response["sys"]["country"])
    except KeyError:
        country.append("NaN")
    try:
        humidity.append(response["main"]["humidity"])
    except KeyError:
        humidity.append("NaN")
    try:
        lat.append(response["coord"]["lat"])
    except KeyError:
        lat.append("NaN")
    try:
        lon.append(response["coord"]["lon"])
    except KeyError:
        lon.append("NaN")
    try:
        max_temp.append(response["main"]["temp_max"])
    except KeyError:
        max_temp.append("NaN")
    try:
        wind_speed.append(response["wind"]["speed"])
    except KeyError:
        wind_speed.append("NaN")

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
# create a dictionary from cities and response values
weather_dict = {
    "city": cities,
    "clouds": clouds,
    "country": country,
    "humidity": humidity,
    "lat": lat,
    "lon": lon,
    "max_temp": max_temp,
    "wind_speed": wind_speed
}

#Convert dictionary to dataframe
weather_data = pd.DataFrame(weather_dict)

#Preview dataframe
weather_data.head()

In [ ]:
#Add today's date to dataframe
date = pd.Timestamp("today").strftime("%m/%d/%Y")
weather_data["date"] = date

#Preview dataframe with new column
weather_data.head()

In [ ]:
#Save dataframe to csv
weather_data.to_csv(r"Results/weather_data.csv")

### Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

#### Latitude vs. Temperature Plot

In [ ]:
#Create scatter plot showing the relationship of latitude and max temperatures for today
plt.scatter(lat, max_temp, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Max Temperature ({date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (Fahrenheit)")
plt.grid(True)
plt.savefig("Results/max_temp-latitude.png")

#### Latitude vs. Humidity Plot

In [ ]:
#Create scatter plot showing the relationship of latitude and humidity for today
plt.scatter(lat, humidity, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Humidity ({date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)
plt.savefig("Results/humidity-latitude.png")

#### Latitude vs. Cloudiness Plot

In [ ]:
#Create scatter plot showing the relationship of latitude and cloudiness for today
plt.scatter(lat, clouds, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Cloudiness ({date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)
plt.savefig("Results/cloudiness-latitude.png")

#### Latitude vs. Wind Speed Plot

In [ ]:
#Create scatter plot showing the relationship of latitude and wind speed for today
plt.scatter(lat, wind_speed, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Windspeed ({date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid(True)
plt.savefig("Results/wind_speed-latitude.png")

### Today's Weather
* As expected, today's max temperatures were highest in cities closest to the equator. As you move away from the equator, the day's max temperatures drops.
* The majority of cities measured saw humidity levels of 70% and above today regardless of their proximity to the equator.
* The wind speed in most cities was low today with the bulk of the cities measured showing wind speeds under 10 mph regardless of their latitudnal position.

In [ ]:
#Create scatter plot showing the relationship of latitude and max temperatures for today
plt.figure(figsize=(6,6))
plt.scatter(lat, max_temp, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Max Temperature ({date})")
plt.xlabel("Latitude")
plt.ylabel("Max Temperature (Fahrenheit)")
plt.grid(True)

plt.savefig("Results/max_temp-latitude-sq.png")

In [ ]:
#Create scatter plot showing the relationship of latitude and humidity for today
plt.figure(figsize=(6,6))
plt.scatter(lat, humidity, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Humidity ({date})")
plt.xlabel("Latitude")
plt.ylabel("Humidity (%)")
plt.grid(True)
plt.savefig("Results/humidity-latitude-sq.png")

In [ ]:
#Create scatter plot showing the relationship of latitude and cloudiness for today
plt.figure(figsize=(6,6))
plt.scatter(lat, clouds, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Cloudiness ({date})")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness (%)")
plt.grid(True)
plt.savefig("Results/cloudiness-latitude-sq.png")

In [ ]:
#Create scatter plot showing the relationship of latitude and wind speed for today
plt.figure(figsize=(6,6))
plt.scatter(lat, wind_speed, marker="o", facecolors="blue", alpha=0.25, edgecolors="black")
plt.title(f"City Latitude vs. Windspeed ({date})")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed (mph)")
plt.grid(True)
plt.savefig("Results/wind_speed-latitude-sq.png")

In [ ]:
weather_data.to_html('weather_data.html')